In [1]:
#import libraries
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import geopy.distance
import re

In [2]:
#load the shape data

tracts = gpd.read_file('merged_CTs.shp')
#tracts.head()
tracts.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3492 entries, 0 to 3491
Data columns (total 13 columns):
STATEFP     3492 non-null object
COUNTYFP    3492 non-null object
TRACTCE     3492 non-null object
GEOID       3492 non-null object
NAME        3492 non-null object
NAMELSAD    3492 non-null object
MTFCC       3492 non-null object
FUNCSTAT    3492 non-null object
ALAND       3492 non-null int64
AWATER      3492 non-null int64
INTPTLAT    3492 non-null object
INTPTLON    3492 non-null object
geometry    3492 non-null object
dtypes: int64(2), object(11)
memory usage: 354.7+ KB


In [21]:
tracts.crs

{'init': 'epsg:4269'}

In [22]:
#load trip data
tracts.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,51,179,010502,51179010502,105.02,Census Tract 105.02,G5020,S,14055402,618196,+38.2809373,-077.4125747,"POLYGON ((-77.458085 38.304298, -77.457764 38...."
1,51,179,010207,51179010207,102.07,Census Tract 102.07,G5020,S,7656808,411682,+38.4803399,-077.4060766,"POLYGON ((-77.433375 38.49089, -77.43328 38.49..."
2,51,179,010406,51179010406,104.06,Census Tract 104.06,G5020,S,6504006,276922,+38.3051765,-077.4335641,"POLYGON ((-77.471358 38.319528, -77.471127 38...."
3,51,179,010103,51179010103,101.03,Census Tract 101.03,G5020,S,5070893,395956,+38.4589777,-077.3928997,"POLYGON ((-77.406522 38.460714, -77.406516 38...."
4,51,179,010301,51179010301,103.01,Census Tract 103.01,G5020,S,65673133,412218,+38.4537694,-077.5712426,"POLYGON ((-77.634309 38.464559, -77.6342799999..."


In [24]:
#import the cleaned DC rides data 
dc_rides = pd.read_csv('dc_trips_raw.csv') 
dc_rides.head()

,Unnamed: 0,OBJECTID,TOTALAMOUNT,PICKUPSTATE,PICKUPZIP,DROPOFFSTATE,DROPOFFZIP,TRIPMILEAGE,TRIPTIME,PICKUP_BLOCK_LATITUDE,PICKUP_BLOCK_LONGITUDE,DROPOFF_BLOCK_LATITUDE,DROPOFF_BLOCK_LONGITUDE,AIRPORT,PICKUPDATETIME_TR,DROPOFFDATETIME_TR,DIST_TIME_RAT,DIST_TIME_RAT_20,TRIPMILEAGE_20,TRIPTIME_20
0,0,4887,8.74,District of Columbia,20036,District of Columbia,20009,1.0,7.0,38.905199,-77.040856,38.915929,-77.031954,N,02/21/2016 00:00,02/21/2016 00:00,0.142857,4,0,3
1,5,4892,7.97,District of Columbia,20004,District of Columbia,20024,1.0,5.0,38.896320,-77.032789,38.887564,-77.023953,N,02/26/2016 12:00,02/26/2016 12:00,0.200000,9,0,1
2,7,4894,6.47,District of Columbia,20002,District of Columbia,20002,1.0,8.0,38.897204,-77.008388,38.901324,-77.002802,N,02/26/2016 13:00,02/26/2016 14:00,0.125000,3,0,4
3,8,4895,10.25,District of Columbia,20004,District of Columbia,20002,2.0,14.0,38.896320,-77.032789,38.897204,-77.008388,N,02/26/2016 13:00,02/26/2016 13:00,0.142857,4,3,10
4,10,4903,7.28,District of Columbia,20036,District of Columbia,20001,1.0,8.0,38.903739,-77.037524,38.900344,-77.022947,N,02/20/2016 20:00,02/20/2016 20:00,0.125000,3,0,4


In [25]:
#split the dataframe into pick up and drop off to prepare for geo-joins:

dc_rides_pickup = dc_rides[['OBJECTID','PICKUP_BLOCK_LATITUDE','PICKUP_BLOCK_LONGITUDE']]

dc_rides_dropoff = dc_rides[['OBJECTID','DROPOFF_BLOCK_LATITUDE','DROPOFF_BLOCK_LONGITUDE']]

In [26]:
#convert ride_data_all to a geopandas dataframe

geometry_pickup = [Point(xy) for xy in zip(dc_rides_pickup['PICKUP_BLOCK_LONGITUDE'], dc_rides_pickup['PICKUP_BLOCK_LATITUDE'])]
crs = {'init': 'epsg:4269'}
pickup_geo = gpd.GeoDataFrame(dc_rides_pickup, crs=crs, geometry=geometry_pickup)

In [27]:
geometry_dropoff = [Point(xy) for xy in zip(dc_rides_dropoff['DROPOFF_BLOCK_LONGITUDE'], dc_rides_dropoff['DROPOFF_BLOCK_LATITUDE'])]
crs = {'init': 'epsg:4269'}
link_geo = gpd.GeoDataFrame(dc_rides_dropoff, crs=crs, geometry=geometry_dropoff)

In [10]:
pickup_geo.head()

,OBJECTID,PICKUP_BLOCK_LATITUDE,PICKUP_BLOCK_LONGITUDE,geometry
0,4887,38.905199,-77.040856,POINT (-77.04085600000001 38.905199)
1,4892,38.896320,-77.032789,POINT (-77.03278900000001 38.89632)
2,4894,38.897204,-77.008388,POINT (-77.008388 38.897204)
3,4895,38.896320,-77.032789,POINT (-77.03278900000001 38.89632)
4,4903,38.903739,-77.037524,POINT (-77.037524 38.903739)


In [ ]:
#spatial join pick-up data with tracts data
geo_pickup = gpd.sjoin(pickup_geo, tracts, how='left', op='intersects')
geo_pickup.head()

In [ ]:
#spatial join drop-off data with tracts data
geo_dropoff = gpd.sjoin(link_geo, tracts, how='left', op='intersects')
geo_dropoff.head()

In [ ]:
#Filter just the necessary columns

geo_pickup = geo_pickup[['GEOID','OBJECTID','PICKUP_BLOCK_LATITUDE','PICKUP_BLOCK_LONGITUDE','geometry']]
geo_dropoff = geo_dropoff[['GEOID','OBJECTID','DROPOFF_BLOCK_LATITUDE','DROPOFF_BLOCK_LONGITUDE','geometry']]

In [ ]:
#convert back to pd dataframe
pickup_CT = pd.DataFrame(geo_pickup)
dropoff_CT = pd.DataFrame(geo_dropoff)

In [ ]:
#join pickup back to dc_rides
dc_rides = dc_rides.merge(pickup_CT,how='left',left_on='OBJECTID',right_on='OBJECTID', inplace=True)

In [ ]:
#join dropoff back to dc_rides
dc_rides = dc_rides.merge(dropoff_CT,how='left',left_on='OBJECTID',right_on='OBJECTID', inplace=True)

In [ ]:
#drop unneeded columns
dc_rides = dc_rides[['']]

In [ ]:
#drop all records more than 6 standard deviations (171mi * 6 = 1026mi)
dc_rides_clean = dc_rides[np.abs(dc_rides['TRIPMILEAGE']- dc_rides['TRIPMILEAGE'].mean()) <= (6*dc_rides['TRIPMILEAGE'].std())]
dc_rides_clean.shape

In [ ]:
#calculate distance based on lat-lon and compare distance to 
dc_rides_clean['geometry_pickup'] = [Point(xy) for xy in zip(dc_rides_pickup['PICKUP_BLOCK_LONGITUDE'], dc_rides_pickup['PICKUP_BLOCK_LATITUDE'])
dc_rides_clean['geometry_dropoff'] = [Point(xy) for xy in zip(dc_rides_dropoff['DROPOFF_BLOCK_LONGITUDE'], dc_rides_dropoff['DROPOFF_BLOCK_LATITUDE'])]

dc_rides_clean.head()

In [ ]:
dc_rides_clean['coordinate_dist'] = geopy.distance.vincenty(dc_rides_clean['geometry_pickup'], dc_rides_clean['geometry_dropoff']).miles